In [1]:
%%capture
%pip install accelerate peft bitsandbytes transformers langchain langchain_community langchain_huggingface nest_asyncio faiss-gpu


In [2]:
import os
import torch
import bitsandbytes
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig

2024-07-19 03:24:31.436438: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-19 03:24:31.436543: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-19 03:24:31.584293: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
from transformers import BitsAndBytesConfig
from huggingface_hub import login

from langchain.text_splitter import CharacterTextSplitter
from langchain_community.document_loaders import TextLoader

from langchain_community.embeddings.huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS

from langchain.prompts import PromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain.chains import LLMChain
from langchain_huggingface import HuggingFacePipeline

import nest_asyncio

In [4]:
from huggingface_hub import login

In [5]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
hug_token = user_secrets.get_secret("huggingface_token")

In [6]:
login(token=hug_token)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [7]:
# Load the pre-trained model and tokenizer from Hugging Face
model_name = 'unsloth/tinyllama-chat' 
model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

config.json:   0%|          | 0.00/719 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/129 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.34k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/438 [00:00<?, ?B/s]

In [9]:
# Function to generate text based on the input question and context
def generate_text(question, context):
    # Define the prompt template with the new instruction
    prompt_template = (f"Context: {context}\n"
                       f"Question: {question}\n"
                       "Please generate the answer to the question based on the context provided which is the retrieved data.\n"
                       "Answer:")
    
    # Tokenize the input prompt
    inputs = tokenizer(prompt_template, return_tensors="pt")
    
    # Generate the output using the model
    output = model.generate(**inputs, max_length=1000, num_return_sequences=1)
    #output = model.generate(**inputs, num_return_sequences=1)
    
    # Decode the generated text
    generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
    
    # Extract and return the generated answer (everything after 'Answer:')
    answer = generated_text.split("Answer:")[1].strip()
    #print(answer)
    return answer

In [10]:
# Example usage
question = "What should be the minimum age of child labour?"
context = "A minimum age of 14: Under s.48 and s.49 of the Child Rights Law 2019, a child under the age of 14 shall not be employed. If free compulsory education requires a child to be in school until after the age of 14, they shall also not be employed. If a child is employable under the Child Rights Law, they may engage in employment in accordance with existing labour laws."
# Generate and print the result
result = generate_text(question, context)
#print(context)
print("Generated Answer:", result)

Generated Answer: The minimum age of child labour is 14 years under the Child Rights Law 2019.
